In [ ]:
from urllib import request
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import decomposition
from sklearn import preprocessing
sns.set()
# Python program to read 
# json file 
 
import json 

# Données photos

In [ ]:
dataphotos = pd.read_json('photos.json', lines=True)

In [ ]:
dataphotos

In [ ]:
dataphotos.to_csv (r'dataphotos.csv', index = False, header=True)


In [ ]:
import pandas
import random

filename = "dataphotos.csv" 
n = sum(1 for line in open(filename))-1  # Calculate number of rows in file
s = n//999  # sample size of 999%
skip = sorted(random.sample(range(1, n+1), n-s))  # n+1 to compensate for header 
dataphotospourcent = pandas.read_csv(filename, skiprows=skip)

In [ ]:
dataphotospourcent

In [ ]:
dataphotospourcent['fichiers'] = 'none'
dataphotospourcent['img_path'] = 'none'
dataphotospourcent['labels'] = 0
dataphotospourcent

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

In [ ]:
import os
from PIL import Image
from PIL import ImageFilter
from PIL import Image, ImageOps

from IPython.display import display # to display images
import cv2
import numpy as np

dico = []

listing = os.listdir("yelp_photos/photos/")
for index, row in dataphotospourcent.iterrows():         
        for file in listing:
            file2 = ( dataphotospourcent.at[index,'photo_id'] + ".jpg")
            row['fichiers'] = file2
            dataphotospourcent.at[index,'fichiers'] = row['fichiers']
            if file == file2 :
                row['img_path'] = file
                dataphotospourcent.at[index,'img_path'] = row['img_path']
                if row['label'] == 'food':
                    row['labels'] = 0
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'drink':
                    row['labels'] = 1
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'outside':
                    row['labels'] = 2
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'inside':
                    row['labels'] = 3
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'menu':
                    row['labels'] = 4
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                    
                    
                    
                #print("-"*100)
                #print("Image originale")
                #print("Fichier:" + "yelp_photos/photos/" + file)
                im = Image.open("yelp_photos/photos/" + file) 
                #im = im.resize((100,100))
                #print("Label:" + dataphotospourcent.at[index,'label'])
                #print(im)
                #display(im)
                # Charger l'image comme matrice de pixels
                img = np.array(Image.open("yelp_photos/photos/" + file))
                img_nparray = Image.fromarray(np.uint8(img))
                # Générer et afficher l'histogramme
                # Pour le normaliser : argument density=True dans plt.hist
                # Pour avoir l'histogramme cumulé : argument cumulative=True
                # print("Histogramme image originale")
                # n, bins, patches = plt.hist(img.flatten(), bins=range(256))
                # plt.show()   
                #print("Image filtrée")
                #display(im.filter(ImageFilter.ModeFilter(size=3)))
                #print("Histogramme image filtrée")
                img2 = np.array(im.filter(ImageFilter.ModeFilter(size=3)))
                #n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
                #plt.show()  
                #print("Image égalisée")
                #display(im.filter(ImageFilter.ModeFilter(size=3)))
                #print("Histogramme image égalisée")
                img3 = np.array(ImageOps.equalize(im.filter(ImageFilter.ModeFilter(size=3)), mask = None))
                #n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
                #plt.show()    
                
                #print("Image avec les features détectées avec ORB")
                gray= cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
                orb = cv2.ORB_create()
                kp = orb.detect(gray,None)
                kp, des = orb.detectAndCompute(img3, None)
                
                #Extract keypoints from each image
                for d in des:
                    dico.append(d)
                img4=cv2.drawKeypoints(gray,kp,img3)
                im1 = Image.fromarray(np.uint8(img4))
                #display(im1)
                #cv2.imwrite(dataphotospourcent.at[index,'photo_id'] + "sift_keypoints" + ".jpg",img4)  
                
 

In [ ]:
dataphotospourcent.to_csv (r'dataphotospourcent.csv', index = False, header=True)

In [ ]:
dataphotospourcent

In [ ]:
import pandas as pd
import os


train_df = dataphotospourcent.drop(["business_id","caption","label","fichiers"], axis=1)
train_df = train_df.rename(columns={"labels": "label"})

train_df.to_csv (r'train_csv.csv', index = False, header=True)

In [ ]:
train_df

In [ ]:
img_path = "yelp_photos/photos/"
train = train_df
labels = train.label.sort_values().unique()

# Clustering

In [ ]:
k = np.size(labels) * 10

batch_size = np.size(os.listdir(img_path)) * 3
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)


In [ ]:
img_nparray

In [ ]:
img.shape

In [ ]:
# Charger l'image sous forme d'une matrice de pixels
img = np.array(img_nparray)

# Générer le bruit gaussien de moyenne nulle et d'écart-type 7 (variance 49)
noise = np.random.normal(0, 7, img.shape)
display(noise)


# Créer l'image bruitée et l'afficher
noisy_img = Image.fromarray(np.uint8(img + noise))
#noisy_img = Image.fromarray(img + noise).convert('L')
#noisy_img.show()
display(noisy_img)

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(noisy_img)
display(noisy_img.filter(ImageFilter.BoxBlur(1)))

In [ ]:
noisy_img2 = noisy_img.filter(ImageFilter.BoxBlur(1))

img1 = np.array(noisy_img)
img2 = np.array(noisy_img2)

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()    


In [ ]:
from PIL import Image, ImageOps

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(noisy_img)
display(ImageOps.equalize(noisy_img.filter(ImageFilter.BoxBlur(1)), mask = None))

In [ ]:

img1 = np.array(img_nparray)
img2 = np.array(noisy_img)
img3 = np.array(ImageOps.equalize(noisy_img.filter(ImageFilter.BoxBlur(1)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.BoxBlur(1)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.BoxBlur(1)), mask = None))

In [ ]:

img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.BoxBlur(1)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.BoxBlur(1)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()    

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.GaussianBlur(radius=1)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.GaussianBlur(radius=1)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.GaussianBlur(radius=1)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.GaussianBlur(radius=1)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()   

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show() 

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.Kernel((3, 3),(1, 1, -1, -1, 9, -1, -2, -3, -1), 1, 0)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.Kernel((3, 3),(1, 1, -1, -1, 9, -1, -2, -3, -1), 1, 0)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.Kernel((3, 3),(1, 1, -1, -1, 9, -1, -2, -3, -1), 1, 0)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.Kernel((3, 3),(1, 1, -1, -1, 9, -1, -2, -3, -1), 1, 0)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show() 

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.RankFilter(1 , 0)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.RankFilter(1 , 0)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.RankFilter(1 , 0)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.RankFilter(1 , 0)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()    

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.MedianFilter(size=3)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.MedianFilter(size=3)), mask = None))


In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.MedianFilter(size=3)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.MedianFilter(size=3)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()   

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()   

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.MinFilter(size=3)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.MinFilter(size=3)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.MinFilter(size=3)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.MinFilter(size=3)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()   

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()  

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.MaxFilter(size=3)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.MaxFilter(size=3)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.MaxFilter(size=3)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.MaxFilter(size=3)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()   

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()   

In [ ]:
from PIL import ImageFilter

# Appliquer le lissage par moyennage (fenêtre de taille 9) et afficher le résultat
display(img_nparray)
display(img_nparray.filter(ImageFilter.ModeFilter(size=3)))
display(ImageOps.equalize(img_nparray.filter(ImageFilter.ModeFilter(size=3)), mask = None))

In [ ]:
img1 = np.array(img_nparray)
img2 = np.array(img_nparray.filter(ImageFilter.ModeFilter(size=3)))
img3 = np.array(ImageOps.equalize(img_nparray.filter(ImageFilter.ModeFilter(size=3)), mask = None))

n, bins, patches = plt.hist(img1.flatten(), bins=range(256))
plt.show()    

n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
plt.show()   

n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
plt.show()   

In [ ]:
import cv2
import numpy as np
#img = cv2.imread('home.jpg')
gray= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
sift = cv2.xfeatures2d.SIFT_create()
kp = sift.detect(gray,None)
img=cv2.drawKeypoints(gray,kp,img)
im1 = Image.fromarray(np.uint8(img))
display(im1)
#cv2.imwrite('sift_keypoints.jpg',img)

In [ ]:
import cv2
import numpy as np
#img = cv2.imread('home.jpg')
gray= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
orb = cv2.ORB_create()
kp = orb.detect(gray,None)
img=cv2.drawKeypoints(gray,kp,img)
im1 = Image.fromarray(np.uint8(img))
display(im1)
#cv2.imwrite('sift_keypoints.jpg',img)

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

In [ ]:
import os
from PIL import Image
from PIL import ImageFilter
from PIL import Image, ImageOps

from IPython.display import display # to display images
import cv2
import numpy as np

dico = []
kmeans.verbose = False
histo_list = []

listing = os.listdir("yelp_photos/photos/")
for index, row in dataphotospourcent.iterrows():         
        for file in listing:
            file2 = ( dataphotospourcent.at[index,'photo_id'] + ".jpg")
            row['fichiers'] = file2
            dataphotospourcent.at[index,'fichiers'] = row['fichiers']
            if file == file2 :
                row['img_path'] = file
                dataphotospourcent.at[index,'img_path'] = row['img_path']
                if row['label'] == 'food':
                    row['labels'] = 0
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'drink':
                    row['labels'] = 1
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'outside':
                    row['labels'] = 2
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'inside':
                    row['labels'] = 3
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                if row['label'] == 'menu':
                    row['labels'] = 4
                    dataphotospourcent.at[index,'labels'] = row['labels']  
                    
                    
                    
                print("-"*100)
                print("Image originale")
                print("Fichier:" + "yelp_photos/photos/" + file)
                im = Image.open("yelp_photos/photos/" + file) 
                #im = im.resize((100,100))
                print("Label:" + dataphotospourcent.at[index,'label'])
                print(im)
                display(im)
                # Charger l'image comme matrice de pixels
                img = np.array(Image.open("yelp_photos/photos/" + file))
                img_nparray = Image.fromarray(np.uint8(img))
                # Générer et afficher l'histogramme
                # Pour le normaliser : argument density=True dans plt.hist
                # Pour avoir l'histogramme cumulé : argument cumulative=True
                print("Histogramme image originale")
                n, bins, patches = plt.hist(img.flatten(), bins=range(256))
                plt.show()   
                print("Image filtrée")
                display(im.filter(ImageFilter.ModeFilter(size=3)))
                print("Histogramme image filtrée")
                img2 = np.array(im.filter(ImageFilter.ModeFilter(size=3)))
                n, bins, patches = plt.hist(img2.flatten(), bins=range(256))
                plt.show()  
                print("Image égalisée")
                display(im.filter(ImageFilter.ModeFilter(size=3)))
                print("Histogramme image égalisée")
                img3 = np.array(ImageOps.equalize(im.filter(ImageFilter.ModeFilter(size=3)), mask = None))
                n, bins, patches = plt.hist(img3.flatten(), bins=range(256))
                plt.show()    
                
                print("Image avec les features détectées avec ORB")
                gray= cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
                orb = cv2.ORB_create()
                kp = orb.detect(gray,None)
                kp, des = orb.detectAndCompute(img3, None)
                
                #Extract keypoints from each image
                for d in des:
                    dico.append(d)
                
                #Creation of the histograms
                histo = np.zeros(k)
                nkp = np.size(kp)
                
                for d in des:
                    idx = kmeans.predict([d])
                    histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

                histo_list.append(histo)                    
    
                img4=cv2.drawKeypoints(gray,kp,img3)
                im1 = Image.fromarray(np.uint8(img4))
                display(im1)
                #cv2.imwrite(dataphotospourcent.at[index,'photo_id'] + "orb_keypoints" + ".jpg",img4)  
                
 

# Training of the neural network

In [ ]:
labels = train.label.sort_values().unique()

In [ ]:
labels

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X = np.array(histo_list)
y = []

# It's a way to convert labels name into an integer
for s in train.label:
    y.append(np.min(np.nonzero(labels == s)))
    
X, y = make_classification(n_samples=100, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

In [ ]:
mlp = MLPClassifier(verbose=True, max_iter=600000)
mlp.fit(X_train, y_train)

In [ ]:
print('Accuracy MLP classifier features obtenue avec ORB')
mlp.score(X_test, y_test)